In [1]:
## mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys
import os
sys.path.append(os.path.abspath('/content/drive/My Drive/image_filtering/'))

Mounted at /content/drive


In [2]:
from __future__ import print_function, division
import torchvision
import torch
from skimage import io, transform
import numpy as np
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import cv2
from math import log10, pi
import time
from google.colab import files

import utils
from datasets import RedFlashDataset
from vgg import Vgg16

from torchvision.utils import save_image

In [3]:
class MFFNet(torch.nn.Module):
    def __init__(self):
        super(MFFNet, self).__init__()
        
        self.conv1 = ConvLayer(4, 32, kernel_size=9, stride=1)
        self.in1 = torch.nn.InstanceNorm2d(32, affine=True)
        self.conv2 = ConvLayer(32, 64, kernel_size=3, stride=2)
        self.in2 = torch.nn.InstanceNorm2d(64, affine=True)
        self.conv3 = ConvLayer(64, 128, kernel_size=3, stride=2)
        self.in3 = torch.nn.InstanceNorm2d(128, affine=True)
        # Residual layers
        self.res1 = ResidualBlock(128)
        self.res2 = ResidualBlock(128)
        self.res3 = ResidualBlock(128)
        self.res4 = ResidualBlock(128)
        self.res5 = ResidualBlock(128)
        self.res6 = ResidualBlock(128)
        self.res7 = ResidualBlock(128)
        self.res8 = ResidualBlock(128)
        self.res9 = ResidualBlock(128)
        self.res10 = ResidualBlock(128)
        self.res11 = ResidualBlock(128)
        self.res12 = ResidualBlock(128)
        self.res13 = ResidualBlock(128)
        self.res14 = ResidualBlock(128)
        self.res15 = ResidualBlock(128)
        self.res16 = ResidualBlock(128)
        
        self.deconv1 = UpsampleConvLayer(128*2, 64, kernel_size=3, stride=1, upsample=2)
        self.in4 = torch.nn.InstanceNorm2d(64, affine=True)
        self.deconv2 = UpsampleConvLayer(64*2, 32, kernel_size=3, stride=1, upsample=2)
        self.in5 = torch.nn.InstanceNorm2d(32, affine=True)
        self.deconv3 = ConvLayer(32*2, 3, kernel_size=9, stride=1)

        self.relu = torch.nn.ReLU()
    
    def forward(self, X):
        o1 = self.relu(self.conv1(X))
        o2 = self.relu(self.conv2(o1))
        o3 = self.relu(self.conv3(o2))

        y = self.res1(o3)
        y = self.res2(y)
        y = self.res3(y)
        y = self.res4(y)
        y = self.res5(y)
        y = self.res6(y)
        y = self.res7(y)
        y = self.res8(y)
        y = self.res9(y)
        y = self.res10(y)
        y = self.res11(y)
        y = self.res12(y)
        y = self.res13(y)
        y = self.res14(y)
        y = self.res15(y)
        y = self.res16(y)
        
        in1 = torch.cat( (y, o3), 1 )
        y = self.relu(self.deconv1(in1))
        in2 = torch.cat( (y, o2), 1 )
        y = self.relu(self.deconv2(in2))
        in3 = torch.cat( (y, o1), 1 )
        y = self.deconv3(in3)
        
        return y

class ConvLayer(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super(ConvLayer, self).__init__()
        reflection_padding = kernel_size // 2
        self.reflection_pad = torch.nn.ReflectionPad2d(reflection_padding)
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride)
    
    def forward(self, x):
        out = self.reflection_pad(x)
        out = self.conv2d(out)
        return out

class ResidualBlock(torch.nn.Module):
    
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = ConvLayer(channels, channels, kernel_size=3, stride=1)
        self.in1 = torch.nn.InstanceNorm2d(channels, affine=True)
        self.conv2 = ConvLayer(channels, channels, kernel_size=3, stride=1)
        self.in2 = torch.nn.InstanceNorm2d(channels, affine=True)
        self.relu = torch.nn.ReLU()
    
    def forward(self, x):
        residual = x
        out = self.relu(self.conv1(x))
        out = self.conv2(out)
        out = out + residual
        return out


class UpsampleConvLayer(torch.nn.Module):
    
    def __init__(self, in_channels, out_channels, kernel_size, stride, upsample=None):
        super(UpsampleConvLayer, self).__init__()
        self.upsample = upsample
        reflection_padding = kernel_size // 2
        self.reflection_pad = torch.nn.ReflectionPad2d(reflection_padding)
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride)
    
    def forward(self, x):
        x_in = x
        if self.upsample:
            x_in = torch.nn.functional.interpolate(x_in, mode='nearest', scale_factor=self.upsample)
        out = self.reflection_pad(x_in)
        out = self.conv2d(out)
        return out

In [4]:
redflash_dataset = RedFlashDataset('/content/drive/My Drive/image_filtering/Deep Learning Final Project/images', True)
train_size = int(0.7 * len(redflash_dataset.fileID))
val_size = len(redflash_dataset.fileID) - train_size
print(train_size, val_size)
train_dataset, val_dataset = torch.utils.data.random_split(redflash_dataset, [train_size, val_size])
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=16,
                                           shuffle=True,
                                           num_workers=0)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                           batch_size=16,
                                           shuffle=True,
                                           num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
imageFilter = MFFNet()
model_name = 'MFF-net'
# imageFilter.load_state_dict(torch.load(f'/content/drive/My Drive/image_filtering/MFF-net-nd.ckpt'))
imageFilter = imageFilter.to(device).float()

# Initializing VGG16 model for perceptual loss
VGG = Vgg16(requires_grad=False)
VGG = VGG.to(device)

1014 435


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [ ]:
num_epochs = 600
learning_rate = 1e-4

criterion_img = nn.MSELoss()
criterion_vgg = nn.MSELoss()

optimizer = torch.optim.Adam(imageFilter.parameters(), lr=learning_rate)
total_step = len(train_loader)

start_epoch = 0

start_time = time.time()
for epoch in range(start_epoch, num_epochs):
    loss_tol = 0
    loss_tol_vgg  = 0
    loss_tol_l2   = 0

    if epoch > 0 and epoch % 20 == 0:
      torch.save(imageFilter.state_dict(), f'/content/drive/My Drive/image_filtering/MFF-net-nd.ckpt')
      files.download('/content/drive/My Drive/image_filtering/MFF-net-nd.ckpt')
    
    if epoch == 300:
        learning_rate = 1e-5
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate
        
    if epoch == 600:
        learning_rate = 1e-6
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate
    
    for i, im in enumerate(train_loader):
        inputs = im[0].float().to(device)
        target = im[1].float().to(device)
        
        outputs = imageFilter(inputs)
        
        loss_l2 = criterion_img( outputs, target )
        
        outputs_n = utils.normalize_ImageNet_stats(outputs)
        target_n  = utils.normalize_ImageNet_stats(target)
        
        feature_o = VGG(outputs_n, 3)
        feature_t = VGG(target_n, 3)
        VGG_loss = []
        for l in range(3+1):
            VGG_loss.append( criterion_vgg(feature_o[l], feature_t[l]) )
        
        loss_vgg = sum(VGG_loss)
        loss = loss_l2 + 0.01*loss_vgg
    
        loss_tol += loss.item()
        
        loss_tol_vgg  += loss_vgg
        loss_tol_l2   += loss_l2
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print ( 'Epoch [{}/{}], Training Loss: {:.4f}, vgg Loss: {:.4f}, L2 Loss: {:.4f}' .format(epoch+1, num_epochs, loss_tol, loss_tol_vgg, loss_tol_l2) )

print("--- %0.4f seconds ---" % (time.time() - start_time)) 
torch.save(imageFilter.state_dict(), f'/content/drive/My Drive/image_filtering/MFF-net-nd.ckpt')
files.download('/content/drive/My Drive/image_filtering/MFF-net-nd.ckpt')

Epoch [1/600], Training Loss: 11.5563, vgg Loss: 933.6595, L2 Loss: 2.2197
Epoch [2/600], Training Loss: 7.2331, vgg Loss: 685.7779, L2 Loss: 0.3753
Epoch [3/600], Training Loss: 5.7115, vgg Loss: 534.6409, L2 Loss: 0.3651
Epoch [4/600], Training Loss: 5.0596, vgg Loss: 476.6694, L2 Loss: 0.2929
Epoch [5/600], Training Loss: 4.6662, vgg Loss: 440.3136, L2 Loss: 0.2631
Epoch [6/600], Training Loss: 4.4699, vgg Loss: 420.1012, L2 Loss: 0.2688
Epoch [7/600], Training Loss: 4.3311, vgg Loss: 408.6003, L2 Loss: 0.2451
Epoch [8/600], Training Loss: 4.0230, vgg Loss: 380.2422, L2 Loss: 0.2206
Epoch [9/600], Training Loss: 4.0003, vgg Loss: 378.7560, L2 Loss: 0.2127
Epoch [10/600], Training Loss: 3.8848, vgg Loss: 367.2649, L2 Loss: 0.2121
Epoch [11/600], Training Loss: 3.7086, vgg Loss: 351.5185, L2 Loss: 0.1934
Epoch [12/600], Training Loss: 3.6544, vgg Loss: 344.5465, L2 Loss: 0.2089
Epoch [13/600], Training Loss: 3.5948, vgg Loss: 341.0427, L2 Loss: 0.1844
Epoch [14/600], Training Loss: 3.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Epoch [21/600], Training Loss: 3.0972, vgg Loss: 296.2436, L2 Loss: 0.1347
Epoch [22/600], Training Loss: 3.0356, vgg Loss: 288.8590, L2 Loss: 0.1470
Epoch [23/600], Training Loss: 2.9808, vgg Loss: 285.3190, L2 Loss: 0.1276
Epoch [24/600], Training Loss: 2.9318, vgg Loss: 280.5728, L2 Loss: 0.1261
Epoch [25/600], Training Loss: 2.9498, vgg Loss: 282.0716, L2 Loss: 0.1291
Epoch [26/600], Training Loss: 2.9815, vgg Loss: 285.4479, L2 Loss: 0.1271
Epoch [27/600], Training Loss: 2.9040, vgg Loss: 279.5026, L2 Loss: 0.1090
Epoch [28/600], Training Loss: 2.8773, vgg Loss: 275.7560, L2 Loss: 0.1197
Epoch [29/600], Training Loss: 2.8390, vgg Loss: 271.7835, L2 Loss: 0.1211
Epoch [30/600], Training Loss: 2.8230, vgg Loss: 271.6785, L2 Loss: 0.1062
Epoch [31/600], Training Loss: 2.7263, vgg Loss: 262.0810, L2 Loss: 0.1055
Epoch [32/600], Training Loss: 2.7307, vgg Loss: 262.2357, L2 Loss: 0.1084
Epoch [33/600], Training Loss: 2.7164, vgg Loss: 260.7654, L2 Loss: 0.1087
Epoch [34/600], Training 

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Epoch [361/600], Training Loss: 1.6844, vgg Loss: 165.2324, L2 Loss: 0.0321
Epoch [362/600], Training Loss: 1.6450, vgg Loss: 161.3398, L2 Loss: 0.0316
Epoch [363/600], Training Loss: 1.6690, vgg Loss: 163.7146, L2 Loss: 0.0319
Epoch [364/600], Training Loss: 1.6750, vgg Loss: 164.3456, L2 Loss: 0.0316
Epoch [365/600], Training Loss: 1.6575, vgg Loss: 162.5703, L2 Loss: 0.0318
Epoch [366/600], Training Loss: 1.6415, vgg Loss: 161.0323, L2 Loss: 0.0311
Epoch [367/600], Training Loss: 1.6447, vgg Loss: 161.3646, L2 Loss: 0.0310
Epoch [368/600], Training Loss: 1.6817, vgg Loss: 164.9697, L2 Loss: 0.0320
Epoch [369/600], Training Loss: 1.6800, vgg Loss: 164.8154, L2 Loss: 0.0319
Epoch [370/600], Training Loss: 1.6406, vgg Loss: 160.9430, L2 Loss: 0.0312
Epoch [371/600], Training Loss: 1.6840, vgg Loss: 165.1725, L2 Loss: 0.0323
Epoch [372/600], Training Loss: 1.7006, vgg Loss: 166.8328, L2 Loss: 0.0323
Epoch [373/600], Training Loss: 1.6688, vgg Loss: 163.6595, L2 Loss: 0.0322
Epoch [374/6

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Epoch [381/600], Training Loss: 1.6163, vgg Loss: 158.5844, L2 Loss: 0.0304
Epoch [382/600], Training Loss: 1.6548, vgg Loss: 162.3361, L2 Loss: 0.0314
Epoch [383/600], Training Loss: 1.6767, vgg Loss: 164.4312, L2 Loss: 0.0324
Epoch [384/600], Training Loss: 1.6550, vgg Loss: 162.3834, L2 Loss: 0.0312
Epoch [385/600], Training Loss: 1.6533, vgg Loss: 162.1615, L2 Loss: 0.0316
Epoch [386/600], Training Loss: 1.6404, vgg Loss: 160.9347, L2 Loss: 0.0310
Epoch [387/600], Training Loss: 1.6754, vgg Loss: 164.3246, L2 Loss: 0.0322
Epoch [388/600], Training Loss: 1.6596, vgg Loss: 162.7931, L2 Loss: 0.0317
Epoch [389/600], Training Loss: 1.6364, vgg Loss: 160.5434, L2 Loss: 0.0310
Epoch [390/600], Training Loss: 1.6701, vgg Loss: 163.8172, L2 Loss: 0.0320


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
imageFil = MFFNet()

model_name = 'MFF-net'
imageFil.load_state_dict( torch.load('/content/drive/My Drive/image_filtering/%s.ckpt'%(model_name)) )
imageFil = imageFil.to(device).float()

In [ ]:
data_root = '/content/drive/My Drive/image_filtering/test'
out_root = '/content/drive/My Drive/image_filtering/output'
if not os.path.exists(out_root):
    os.mkdir(out_root)

for seq in range(1,6):
    # input rgb image is obtained from demosaicing the raw (no other manipulation)
    # saved in 16-bit TIFF image
    file = ('rgb_%s.tiff' % (seq) )
    filename = os.path.join( data_root, file )
    inputs = io.imread(filename) / 65535

    file = ('guide_%s.bmp' % (seq) )
    filename = os.path.join( data_root, file )
    guided = io.imread(filename) / 255

    guided = guided[:,:,0]+guided[:,:,1]+guided[:,:,2]
    inputs = (inputs*80)**0.4
    inputs = np.concatenate((inputs, guided[:,:,None]), 2)
    inputs = np.transpose(inputs,(2,0,1))
    inputs = torch.from_numpy(inputs)
    inputs = inputs[None,:,:,:].float()

    with torch.no_grad():
        inputs = inputs.to(device) 
        outputs = imageFil(inputs)
    outputs[outputs>1] = 1
    outputs[outputs<0] = 0    

    # the parameter for color balance and brightness should be tuned for different scenes
    outputs[0,0,:,:] = outputs[0,0,:,:]*1.1*1.5
    outputs[0,1,:,:] = outputs[0,1,:,:]*1*1.5
    outputs[0,2,:,:] = outputs[0,2,:,:]*1.5*1.5

    save_image(outputs[0,:,:,:], '%s/out_%s.png' % (out_root, seq))
    save_image(inputs[0,0:3,:,:], '%s/inp_%s.png' % (out_root, seq))